In [6]:
import sklearn.metrics as skm
import sys
import numpy as np
from sklearn.metrics import roc_curve
from sklearn.metrics import RocCurveDisplay
import matplotlib.pyplot as plt

ssl_features_train = np.load("./old_but_full_features/self_supervised_cifar100/features_train2.npy")
ssl_labels_train = np.load("./old_but_full_features/self_supervised_cifar100/labels_train2.npy")
ssl_features_test = np.load("./old_but_full_features/self_supervised_cifar100/features_test2.npy")
ssl_features_ood = np.load("./old_but_full_features/self_supervised_cifar100/features_ood2.npy")


sp_features_train = np.load("./old_but_full_features/supervised_cifar100/features_train2.npy")
sp_labels_train = np.load("./old_but_full_features/supervised_cifar100/labels_train2.npy")
sp_features_test = np.load("./old_but_full_features/supervised_cifar100/features_test2.npy")
sp_features_ood = np.load("./old_but_full_features/supervised_cifar100/features_ood2.npy")


In [27]:
import os
import time
# from util.args_loader import get_args
# from util import metrics
import torch
import faiss
import numpy as np

import sklearn.metrics as skm
import sys
from sklearn.metrics import roc_curve
from sklearn.metrics import RocCurveDisplay
import matplotlib.pyplot as plt

def get_roc_sklearn(xin, xood):
    labels = [0] * len(xin) + [1] * len(xood)
    data = np.concatenate((xin, xood))
    auroc = skm.roc_auc_score(labels, data)
    return auroc

def dnorm(ftrain, ftest, food):
    
    print("ftrain size is:", ftrain.shape)
    ftrain /= np.linalg.norm(ftrain, axis=-1, keepdims=True) + 1e-10
    ftest /= np.linalg.norm(ftest, axis=-1, keepdims=True) + 1e-10
    food /= np.linalg.norm(food, axis=-1, keepdims=True) + 1e-10

    m, s = np.mean(ftrain, axis=0, keepdims=True), np.std(ftrain, axis=0, keepdims=True)
    print("mean value is:", np.mean(m))
    print("std value is:", np.mean(s))
#     m2, s2 = np.mean(ftest, axis=0, keepdims=True), np.std(ftest, axis=0, keepdims=True)
#     m3, s3 = np.mean(food, axis=0, keepdims=True), np.std(food, axis=0, keepdims=True)

    ftrain = (ftrain - m) / (s + 1e-10)
    ftest = (ftest - m) / (s + 1e-10)
    food = (food - m) / (s + 1e-10)
    
    return ftrain, ftest, food

In [28]:
def cal_metric(known, novel, method=None):
    tp, fp, fpr_at_tpr95 = get_curve(known, novel, method)
    results = dict()
    mtypes = ['FPR', 'AUROC', 'DTERR', 'AUIN', 'AUOUT']

    results = dict()

    # FPR
    mtype = 'FPR'
    results[mtype] = fpr_at_tpr95

    # AUROC
    mtype = 'AUROC'
    tpr = np.concatenate([[1.], tp/tp[0], [0.]])
    fpr = np.concatenate([[1.], fp/fp[0], [0.]])
    results[mtype] = -np.trapz(1.-fpr, tpr)

    # DTERR
    mtype = 'DTERR'
    results[mtype] = ((tp[0] - tp + fp) / (tp[0] + fp[0])).min()

    # AUIN
    mtype = 'AUIN'
    denom = tp+fp
    denom[denom == 0.] = -1.
    pin_ind = np.concatenate([[True], denom > 0., [True]])
    pin = np.concatenate([[.5], tp/denom, [0.]])
    results[mtype] = -np.trapz(pin[pin_ind], tpr[pin_ind])

    # AUOUT
    mtype = 'AUOUT'
    denom = tp[0]-tp+fp[0]-fp
    denom[denom == 0.] = -1.
    pout_ind = np.concatenate([[True], denom > 0., [True]])
    pout = np.concatenate([[0.], (fp[0]-fp)/denom, [.5]])
    results[mtype] = np.trapz(pout[pout_ind], 1.-fpr[pout_ind])

    return results

def get_curve(known, novel, method=None):
    tp, fp = dict(), dict()
    fpr_at_tpr95 = dict()

    known.sort()
    novel.sort()

    end = np.max([np.max(known), np.max(novel)])
    start = np.min([np.min(known),np.min(novel)])

    all = np.concatenate((known, novel))
    all.sort()

    num_k = known.shape[0]
    num_n = novel.shape[0]

    if method == 'row':
        threshold = -0.5
    else:
        threshold = known[round(0.05 * num_k)]

    tp = -np.ones([num_k+num_n+1], dtype=int)
    fp = -np.ones([num_k+num_n+1], dtype=int)
    tp[0], fp[0] = num_k, num_n
    k, n = 0, 0
    for l in range(num_k+num_n):
        if k == num_k:
            tp[l+1:] = tp[l]
            fp[l+1:] = np.arange(fp[l]-1, -1, -1)
            break
        elif n == num_n:
            tp[l+1:] = np.arange(tp[l]-1, -1, -1)
            fp[l+1:] = fp[l]
            break
        else:
            if novel[n] < known[k]:
                n += 1
                tp[l+1] = tp[l]
                fp[l+1] = fp[l] - 1
            else:
                k += 1
                tp[l+1] = tp[l] - 1
                fp[l+1] = fp[l]

    j = num_k+num_n-1
    for l in range(num_k+num_n-1):
        if all[j] == all[j-1]:
            tp[j] = tp[j+1]
            fp[j] = fp[j+1]
        j -= 1

    fpr_at_tpr95 = np.sum(novel > threshold) / float(num_n)

    return tp, fp, fpr_at_tpr95

In [29]:
ssl_features_train,  ssl_features_test, ssl_features_ood= dnorm(ssl_features_train, ssl_features_test, ssl_features_ood)
sp_features_train,  sp_features_test, sp_features_ood= dnorm(sp_features_train, sp_features_test, sp_features_ood)

ftrain size is: (50000, 2048)
mean value is: 1.2514904e-06
std value is: 0.022090783
ftrain size is: (50000, 2048)
mean value is: -0.00021042174
std value is: 0.02207962


In [34]:
sp_din = np.load("./old_but_full_features/supervised_cifar100/sl_test_score.npy")
sp_dout = np.load("./old_but_full_features/supervised_cifar100/sl_ood_score.npy")
print('AUROC is:',  get_roc_sklearn(sp_din, sp_dout))

AUROC is: 0.8796951750000002


In [43]:
index = faiss.IndexFlatL2(ssl_features_train.shape[1])
print("feature shape is:", ssl_features_train.shape)
index.add(ssl_features_train)

K = 50
D, _ = index.search(ssl_features_test, K)
print("D shape is:", D.shape)
# print(D[:3])
scores_in = -D[:,-1]
# print(scores_in[:3])
print(scores_in.shape)
#     all_results = []
#     all_score_ood = []
#     for ood_dataset, food in food_all.items():
D, _ = index.search(ssl_features_ood, K)
scores_ood_test = -D[:,-1]
#     all_score_ood.extend(scores_ood_test)
results = cal_metric(scores_in, scores_ood_test)
# results = cal_metric(-sp_din, -sp_dout)
#     all_results.append(results)

#     metrics.print_all_results(results, args.out_datasets, f'KNN k={K}')
#     print()
print(results)

feature shape is: (50000, 2048)
D shape is: (10000, 50)
(10000,)
{'FPR': 0.5369, 'AUROC': 0.90142816, 'DTERR': 0.1681, 'AUIN': 0.9145815684220249, 'AUOUT': 0.8771856485093654}


In [65]:




def get_scores_multi_cluster(ftrain, ftest, food, ypred):
    print("using multiple clusters")
    xc = [ftrain[ypred == i] for i in np.unique(ypred)]
    

    din = [
        np.sum(
            (ftest - np.mean(x, axis=0, keepdims=True))
            * (
                np.linalg.pinv(np.cov(x.T, bias=True)).dot(
                    (ftest - np.mean(x, axis=0, keepdims=True)).T
                )
            ).T,
            axis=-1,
        )
        for x in xc
    ]
    dood = [
        np.sum(
            (food - np.mean(x, axis=0, keepdims=True))
            * (
                np.linalg.pinv(np.cov(x.T, bias=True)).dot(
                    (food - np.mean(x, axis=0, keepdims=True)).T
                )
            ).T,
            axis=-1,
        )
        for x in xc
    ]
#     print("din shape is looking like:", din)
#     din_index = np.argmin(din, axis = 0)
#     din_2 = np.partition(din, 2, axis = 1)[:2]
#     din_2 = np.sort(din, axis = 0)[:5, :]
#     din_min5_idx = np.argsort(din, axis = 0)[:5, :]
#     print(din_2)
#     print(din_2_idx)
#     din = np.min(din, axis=0)
    din_5 = np.sort(din, axis = 0)[0, :]
    print("din_5 shape is:", din_5.shape)
#     din = np.mean(din_5, axis = 0)
#     print("din shape is:", din.shape)
    
    
#     dood_index = np.argmin(dood, axis = 0)
#     dood_min5_idx = np.argsort(dood, axis = 0)[:5, :]
#     dood = np.min(dood, axis=0)
    dood_5 = np.sort(dood, axis = 0)[0, :]
    print("dood_5 shape is:", dood_5.shape)
#     dood = np.mean(dood_5, axis = 0)
#     print("dood shape is:", dood.shape)

#     return din, dood, din_index, dood_index
    return din_5, dood_5

ssl_din, ssl_dout = get_scores_multi_cluster(ssl_features_train, ssl_features_test, ssl_features_ood, ssl_labels_train)


# index = faiss.IndexFlatL2(ssl_features_train.shape[1])
# print("feature shape is:", ssl_features_train.shape)
# index.add(ssl_features_train)

# K = 50
# D, _ = index.search(ssl_features_test, K)
# print("D shape is:", D.shape)
# # print(D[:3])
# scores_in = -D[:,-1]
# # print(scores_in[:3])
# print(scores_in.shape)
# #     all_results = []
# #     all_score_ood = []
# #     for ood_dataset, food in food_all.items():
# D, _ = index.search(ssl_features_ood, K)
# scores_ood_test = -D[:,-1]
#     all_score_ood.extend(scores_ood_test)
results = cal_metric(-ssl_din, -ssl_dout)
# results = cal_metric(-sp_din, -sp_dout)
#     all_results.append(results)

#     metrics.print_all_results(results, args.out_datasets, f'KNN k={K}')
#     print()
print(results)

using multiple clusters
din_5 shape is: (10000,)
dood_5 shape is: (10000,)
{'FPR': 0.6453, 'AUROC': 0.880571345, 'DTERR': 0.1905, 'AUIN': 0.9019428211379042, 'AUOUT': 0.8497317435628178}


In [88]:
# calculate score for combined feature

ssl_features_train = np.load("./old_but_full_features/self_supervised_cifar100/features_train2.npy")
ssl_labels_train = np.load("./old_but_full_features/self_supervised_cifar100/labels_train2.npy")
ssl_features_test = np.load("./old_but_full_features/self_supervised_cifar100/features_test2.npy")
ssl_features_ood = np.load("./old_but_full_features/self_supervised_cifar100/features_ood2.npy")


sp_features_train = np.load("./old_but_full_features/supervised_cifar100/features_train2.npy")
sp_labels_train = np.load("./old_but_full_features/supervised_cifar100/labels_train2.npy")
sp_features_test = np.load("./old_but_full_features/supervised_cifar100/features_test2.npy")
sp_features_ood = np.load("./old_but_full_features/supervised_cifar100/features_ood2.npy")

import matplotlib.pyplot as plt

def dnorm(ftrain, ftest, food):
    
    print("ftrain size is:", ftrain.shape)
    ftrain /= np.linalg.norm(ftrain, axis=-1, keepdims=True) + 1e-10
    ftest /= np.linalg.norm(ftest, axis=-1, keepdims=True) + 1e-10
    food /= np.linalg.norm(food, axis=-1, keepdims=True) + 1e-10

    m, s = np.mean(ftrain, axis=0, keepdims=True), np.std(ftrain, axis=0, keepdims=True)
    print("mean value is:", np.mean(m))
    print("std value is:", np.mean(s))
#     m2, s2 = np.mean(ftest, axis=0, keepdims=True), np.std(ftest, axis=0, keepdims=True)
#     m3, s3 = np.mean(food, axis=0, keepdims=True), np.std(food, axis=0, keepdims=True)

    ftrain = (ftrain - m) / (s + 1e-10)
    ftest = (ftest - m) / (s + 1e-10)
    food = (food - m) / (s + 1e-10)
#     print("ftrain shape is:", ftrain.shape)
#     ftrain = (ftrain - m) / (s + 1e-10)
#     ftest = (ftrain - m2) / (s2 + 1e-10)
#     food = (ftrain - m3) / (s3 + 1e-10)

#     plt.figure(0).clf()
#     plt.plot(food[960])
#     plt.hist(ftrain[60], density=True, bins=100)
#     plt.show()
    
    
    return ftrain, ftest, food

ssl_features_train,  ssl_features_test, ssl_features_ood= dnorm(ssl_features_train, ssl_features_test, ssl_features_ood)
sp_features_train,  sp_features_test, sp_features_ood= dnorm(sp_features_train, sp_features_test, sp_features_ood)

# alpha = 0.5
# fl_features_train = alpha * ssl_features_train + (1-alpha) * sp_features_train
# fl_features_test = alpha * ssl_features_test + (1-alpha) * sp_features_test
# fl_features_ood = alpha * ssl_features_ood + (1-alpha) * sp_features_ood

fl_features_train = np.concatenate((ssl_features_train, sp_features_train), axis = 1)
fl_features_test = np.concatenate((ssl_features_test, sp_features_test), axis = 1)
fl_features_ood = np.concatenate((ssl_features_ood, sp_features_ood), axis = 1)

print("new fl_feature_train size is:", fl_features_train.shape)

ftrain size is: (50000, 2048)
mean value is: 0.010148109
std value is: 0.018416166
ftrain size is: (50000, 2048)
mean value is: 0.011190266
std value is: 0.012594055
new fl_feature_train size is: (50000, 4096)


In [91]:
index = faiss.IndexFlatL2(sp_features_train.shape[1])
print("feature shape is:", sp_features_train.shape)
index.add(sp_features_train)

K = 50
D, _ = index.search(sp_features_test, K)
print("D shape is:", D.shape)
# print(D[:3])
# scores_in = -D[:,-1]
scores_in = -np.mean(D, axis = 1)
# print(scores_in[:3])
print(scores_in.shape)
#     all_results = []
#     all_score_ood = []
#     for ood_dataset, food in food_all.items():
D, _ = index.search(sp_features_ood, K)
# scores_ood_test = -D[:,-1]
scores_ood_test = -np.mean(D, axis = 1)
#     all_score_ood.extend(scores_ood_test)
results = cal_metric(scores_in, scores_ood_test)

# results = cal_metric(-sp_din, -sp_dout)
#     all_results.append(results)

#     metrics.print_all_results(results, args.out_datasets, f'KNN k={K}')
#     print()
print(results)
# print('AUROC is:',  get_roc_sklearn(-scores_in, -sp_dout))

feature shape is: (50000, 2048)
D shape is: (10000, 50)
(10000,)
{'FPR': 0.5008, 'AUROC': 0.892702345, 'DTERR': 0.18015, 'AUIN': 0.895157608416734, 'AUOUT': 0.8790974995036022}
